In [ ]:
# #add the following as the first line of every py script (include the '#')
# #this will make it so that I dont have to type 'python' before running the script

#!/Users/dianaha/anaconda3/bin/python

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split

from keras.models import load_model
from keras import backend as K
from keras import optimizers
from keras import losses
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.layers import Convolution2D, MaxPooling2D

import tensorflow as tf

import os
from PIL import Image
import cv2
import h5py
import glob

Using TensorFlow backend.


In [2]:
# !pip3 install --upgrade "https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.9.0-py3-none-any.whl"

^C
Traceback (most recent call last):
  File "/Users/dianaha/anaconda3/bin/pip3", line 7, in <module>
    from pip._internal import main
  File "/Users/dianaha/anaconda3/lib/python3.6/site-packages/pip/_internal/__init__.py", line 42, in <module>
    from pip._internal import cmdoptions
  File "/Users/dianaha/anaconda3/lib/python3.6/site-packages/pip/_internal/cmdoptions.py", line 16, in <module>
    from pip._internal.index import (
  File "/Users/dianaha/anaconda3/lib/python3.6/site-packages/pip/_internal/index.py", line 25, in <module>
    from pip._internal.download import HAS_TLS, is_url, path_to_url, url_to_path
  File "/Users/dianaha/anaconda3/lib/python3.6/site-packages/pip/_internal/download.py", line 40, in <module>
    from pip._internal.utils.logging import indent_log
  File "/Users/dianaha/anaconda3/lib/python3.6/site-packages/pip/_internal/utils/logging.py", line 9, in <module>
    from pip._internal.utils.misc import ensure_dir
  File "/Users/dianaha/anaconda3/lib/python

# Prep train and test data for modeling

In [2]:
train = pd.read_csv('./sign-language-mnist/sign_mnist_train.csv')

test = pd.read_csv('./sign-language-mnist/sign_mnist_test.csv')

In [3]:
X = train.drop(['label'], axis=1).values
y = train['label'].values

testset_X = test.drop(['label'], axis=1).values
testset_y = test['label'].values

In [4]:
# change shape of data
X = X.reshape(X.shape[0], 28, 28, 1)

testset_X = testset_X.reshape(testset_X.shape[0], 28, 28, 1)

In [5]:
# change the data from integer to float
X = X.astype('float32')

testset_X = testset_X.astype('float32')

In [6]:
# scale data
X /= 255

testset_X /= 255

In [7]:
# one hot encode our y variables of our data
y = to_categorical(y)

testset_y = to_categorical(testset_y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Prep image for prediction (see below for df input)

In [11]:
img = cv2.imread("flip_color_0_0040.jpg", 0)
re = cv2.resize(img, (28, 28))
arr = np.asarray(re).ravel()

In [12]:
img_df = pd.DataFrame([arr])

In [13]:
img_df

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,42,48,43,44,45,37,32,38,39,36,...,91,92,80,20,144,230,243,218,225,223


In [14]:
img2 = cv2.imread("b.jpg", 0)
re2 = cv2.resize(img2, (28, 28))
arr2 = np.asarray(re2).ravel()
img_df = img_df.append([arr2], ignore_index=True)

In [15]:
img_df

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,42,48,43,44,45,37,32,38,39,36,...,91,92,80,20,144,230,243,218,225,223
1,198,197,198,198,199,200,200,200,201,200,...,54,51,54,52,49,43,105,75,64,68


In [16]:
img_df = img_df.values.reshape(img_df.shape[0], 28, 28, 1)

In [17]:
img_df = img_df.astype('float32')

In [18]:
img_df /= 255

# Prep df for predictions (done on sample data)

In [8]:
images = glob.glob("./my_test/*.jpg", recursive=True)

In [9]:
images

['./my_test/h.jpg',
 './my_test/bh.jpg',
 './my_test/as.jpg',
 './my_test/ch.jpg',
 './my_test/l.jpg',
 './my_test/oh.jpg',
 './my_test/ls.jpg',
 './my_test/o.jpg',
 './my_test/eh.jpg',
 './my_test/hs.jpg',
 './my_test/c.jpg',
 './my_test/a.jpg',
 './my_test/bd.jpg',
 './my_test/e.jpg']

In [10]:
# create empty df and list for labels
pixel_df = pd.DataFrame()


In [11]:
# add images as arrays to train_df
start = 0

for i in images:
#     img = Image.open(i)
#     arr = np.asarray(img).ravel()
#     pixel_df.loc[start] = arr
#     start += 1   
    img = cv2.imread(i, 0)
    re = cv2.resize(img, (28, 28))
    arr = np.asarray(re).ravel()
    pixel_df = pixel_df.append([arr], ignore_index=True)

In [12]:
pixel_df = pixel_df.values.reshape(pixel_df.shape[0], 28, 28, 1)
pixel_df = pixel_df.astype('float32')
pixel_df /= 255

In [13]:
pixel_df.shape

(14, 28, 28, 1)

In [16]:
# # save df
# pixel_df.to_csv("pixel_df.csv", index=False)

# Model

In [9]:
%%time
model = Sequential()
model.add(Convolution2D(filters =32,
                       kernel_size=5,
                       activation='relu',
                       input_shape = (28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2),
                         strides=2,
                        padding='same'))
model.add(Dropout(0.3))

model.add(Convolution2D(filters=64,
                      kernel_size=5,
                      activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),
                        strides=2,
                        padding='same'))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(60, activation='relu'))
model.add(Dense(25, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
                               
model.fit(X_train, y_train, epochs=20, batch_size=1000, verbose=0, validation_data=(X_test, y_test))

# score = model.evaluate(testset_X, testset_y, verbose=0)

# pred_proba = model.predict(img_df)
# pred_class = model.predict_classes(pixel_df)
pred_class = model.predict_classes(testset_X)


# model.save("model3.h5")

# print('test loss: ', score[0])
# print('test accuracy: ', score[1])
# print('Predicted Probability: ', pred_proba)
print('Predicted Class: ', pred_class)

# model3 acc: 94.7%, loss: 15.4%

Predicted Class:  [ 6  5 10 ...  2  4  2]
CPU times: user 15min 55s, sys: 2min 3s, total: 17min 59s
Wall time: 5min 30s


# Reading in model1 and making predictions

In [1]:
model1 = load_model("model1.h5")

NameError: name 'load_model' is not defined

In [ ]:
%%time
score1 = model1.evaluate(testset_X, testset_y, verbose=0)
print('test loss: ', score1[0])
print('test accuracy: ', score1[1])

In [ ]:
%%time
preds1 = model1.predict_classes(pixel_df)

In [ ]:
preds1

# Look at what is being misclassified
### NOTE: Focus on Model 1 as it performed the best

In [21]:
review = pd.DataFrame()
review

""


In [23]:
review['actual'] = test['label']
review['predicted'] = pred_class
review

,actual,predicted
0,6,6
1,5,5
2,10,10
3,0,0
4,3,3
5,21,21
6,10,10
7,14,14
8,3,3
9,7,7


In [36]:
missclass = review[review['actual'] != review['predicted']]

In [33]:
review['actual'][review['actual'] != review['predicted']].count()

382

In [52]:
missclass['predicted'][missclass['actual'] ==24].value_counts()

15    3
Name: predicted, dtype: int64

# Reading in model2 and making predictions

In [122]:
model2 = load_model("model2.h5")

In [123]:
%%time
score2 = model2.evaluate(testset_X, testset_y, verbose=0)
print('test loss: ', score2[0])
print('test accuracy: ', score2[1])

test loss:  0.16777871462626345
test accuracy:  0.9461795872838817
CPU times: user 4.8 s, sys: 381 ms, total: 5.18 s
Wall time: 2.39 s


In [124]:
%%time
preds2 = model2.predict_classes(pixel_df)

CPU times: user 167 ms, sys: 4.72 ms, total: 172 ms
Wall time: 164 ms


In [125]:
preds2

array([15,  5, 15,  6,  8,  7, 15,  7, 16,  7,  2, 15, 22, 15])

# Reading in model3 and making predictions

In [126]:
model3 = load_model("model3.h5")

In [127]:
%%time
score3 = model3.evaluate(testset_X, testset_y, verbose=0)
print('test loss: ', score3[0])
print('test accuracy: ', score3[1])

test loss:  0.1538031328006191
test accuracy:  0.9471556051310652
CPU times: user 4.99 s, sys: 262 ms, total: 5.25 s
Wall time: 1.85 s


In [128]:
%%time
preds3 = model3.predict_classes(pixel_df)

CPU times: user 187 ms, sys: 7.08 ms, total: 194 ms
Wall time: 190 ms


In [129]:
preds3

array([15,  5, 10,  6,  8,  7, 15, 24, 16, 15,  2, 15, 10, 15])

# Convert numerical predictions to string

In [ ]:
letter_dict = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E", 5:"F", 
               6:"G", 7:"H", 8:"I", 10:"K", 11:"L", 12:"M", 
               13:"N", 14:"O", 15:"P", 16:"Q", 17:"R", 18:"S",
               19:"T", 20:"U", 21:"V", 22:"W", 23:"X", 24:"Y"}

In [29]:
translation = "".join(list(map(lambda key: letter_dict[key], preds)))

In [31]:
print(translation)

PFKGIHPYQPCPKP


# WAFFLES: Beginning to End

In [52]:
from PIL import Image

In [73]:
# create WAFFLES images for model input
temp = testset_X[38]
temp=np.reshape(temp,(28,28))
temp=temp*255
im = Image.fromarray(temp).convert(mode='L')
# im.save("s.png")

**NOTE:**    
Test images for WAFFLES were taken from ```testset_X[i]```, where ```i``` is item number:
- 16 for W
- 3 for A
- 1 for F
- 7140 for F
- 459 for L
- 15 for E
- 38 for S

In [99]:
# get file paths for each png image in the folder
waffs_imgs = sorted(glob.glob("./WAFFLES/*.png", recursive=True))
# recursive=True tells it to go into the folder repeatedly to find each file
# sorted(glob.glob('*.png')) <--sorts by filename
# sorted(glob.glob('*.png'), key=os.path.getmtime) <--sorts by modification time
# sorted(glob.glob('*.png'), key=os.path.getsize) <--sorts by size

waffs_imgs

['./WAFFLES/1_w.png',
 './WAFFLES/2_a.png',
 './WAFFLES/3_f.png',
 './WAFFLES/4_ff.png',
 './WAFFLES/5_l.png',
 './WAFFLES/6_e.png',
 './WAFFLES/7_s.png']

In [100]:
# create empty df and list for labels
waffs_df = pd.DataFrame()

In [101]:
# add images as arrays to waffs_df
start = 0

for i in waffs_imgs:
#     img = Image.open(i)
#     arr = np.asarray(img).ravel()
#     pixel_df.loc[start] = arr
#     start += 1   
    img = cv2.imread(i, 0)
    re = cv2.resize(img, (28, 28))
    arr = np.asarray(re).ravel()
    waffs_df = waffs_df.append([arr], ignore_index=True)

In [102]:
# format the data in order to load into the model
waffs_df = waffs_df.values.reshape(waffs_df.shape[0], 28, 28, 1)
waffs_df = waffs_df.astype('float32')
waffs_df /= 255

In [103]:
# load keras model
model1 = load_model("model1.h5")

In [104]:
%%time
# make predictions; ouput is a number
waffs_preds = model.predict_classes(waffs_df)

CPU times: user 6.16 ms, sys: 1.62 ms, total: 7.78 ms
Wall time: 5.14 ms


In [105]:
# dictionary used to convert numerical values to English alphabet
letter_dict = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E", 5:"F", 
               6:"G", 7:"H", 8:"I", 10:"K", 11:"L", 12:"M", 
               13:"N", 14:"O", 15:"P", 16:"Q", 17:"R", 18:"S",
               19:"T", 20:"U", 21:"V", 22:"W", 23:"X", 24:"Y"}

In [108]:
# change output from numbers to a single string of alphabets
translation = "".join(list(map(lambda key: letter_dict[key], waffs_preds)))
print(translation)

WAFFLES


In [116]:
# create function for cell above to return value when running python script
def translate_num2let(waffs_preds):
    translation = "".join(list(map(lambda key: letter_dict[key], waffs_preds)))
    return translation

In [117]:
translate_num2let(waffs_preds)

'WAFFLES'

# Input array, Output Image

In [82]:
from PIL import Image
temp = test_r.iloc[0]
temp=np.reshape(temp,(28,28))
temp=temp*255
im = Image.fromarray(temp).convert(mode='L') # L for black and white
# im.save("pic1.jpg")

TypeError: Cannot handle this data type

In [33]:
from PIL import Image
temp = img_df[1]
temp=np.reshape(temp,(28,28))
temp=temp*255
im = Image.fromarray(temp).convert('RGB') # RGB for color
# im.save("img_df1.jpg")